In [ ]:
# import libraries
import numpy as np 
import pandas as pd
import statsmodels.stats.power as smp
import matplotlib.pyplot as plt

In [ ]:
# let scientists compete against each other throughout their entire lifespan
def competition(lifeSpan, sampleSize, scientistPerQuestionMax, startupCost, sampleCost, ExpDistributionShape, scoopedCost, negativeResultCost, limit, newQuestionCost, randomChance, chance):
    oneYear = lifeSpan/20 # definition of one year in time units, career of scientist takes 20 years
    scientistID = list(np.arange(populationSize)) # list with scientist IDs
    amountOfQuestions = round((lifeSpan / (startupCost + minSampleSize)) * populationSize + populationSize) # generates number of questions
    questionID = list(np.arange(amountOfQuestions)) # list with question IDs
    effectSizeQuestion = list(np.round(np.array(np.random.exponential(1/ExpDistributionShape, size=amountOfQuestions)), 1)) # list with effect sizes drawn from a uniform distribution
    drawerQ = list(np.array(np.zeros(shape=[populationSize, 0]),np.int32)) # integer list of completed question IDs indexed by scientist ID
    drawerR = list(np.array(np.zeros(shape=[populationSize, 0]),np.bool_)) # boolean list of completed question results indexed by scientist ID
    drawerP = []
    drawerU = []
    
   
    
    if randomChance == True:
        chanceNewQ = list(np.random.randint(0, 10, size=populationSize))
    else:
        chanceNewQ = chance
    
    # create scientist dataframe
    d1 = {'scientistID' : scientistID, 'sampleSize' : sampleSize, 'questionID' : 0, 'publications' : 0, 'payoff' : 0.0,'chance': chanceNewQ }
    scientistDataFrame = pd.DataFrame(data=d1)

    # create results dataframe
    d2 = {'questionID' : questionID, 'scientistID' : -1, 'sampleSize' : 0, 'effectSize': effectSizeQuestion,'result' : 0, 'published':None}
    resultsDataFrame = pd.DataFrame(data=d2)
    
    
    # question trackers
    questionIDsPublished = [] # list of questions that have been published
    questionIDsWorked = [] # list of questions that have been completed
    
    # time (cost) trackers
    timePeriod = 1 #start
    timeCost = list(scientistDataFrame['sampleSize'] * sampleCost + startupCost) # real-time tracker
    timeCostBaseline = list(scientistDataFrame['sampleSize'] * sampleCost + startupCost) # baseline tracker
    extraTimeCost = [0] * populationSize

    # assign scientists to questions
    scientistDataFrame['questionID'] = np.repeat(questionID, scientistPerQuestionMax)[:len(scientistDataFrame['scientistID'])] 

    # create a list that keeps track of amount of scientists on a question
    filledQuestions = pd.crosstab(index=scientistDataFrame['questionID'], columns="count") #creates df with questionid and nr of scientis working on it
    emptyQuestions = (amountOfQuestions - len(filledQuestions)) * [0] # creates a list with 0's with len of unused questions
    emptyQuestions = pd.DataFrame(emptyQuestions, columns=["count"]) # df of question id and nr of scientis working on 
    questionDataFrame = filledQuestions.append(emptyQuestions, ignore_index= True) # combine the two df's 
    scientistPerQuestion = list(questionDataFrame['count']) #create a list of the count df

    
    yearProgress = oneYear # time to next year tracker
    unpubQ = questionID # list of all questions that have been worked on
    
    while(timePeriod < lifeSpan):
            
        # calculate time to next event: end of year / completed research / end of lifespan
        timeToNextEvent = min(yearProgress, min( min(timeCost), lifeSpan - timePeriod)) 
        
        # update time trackers
        if limit:
            yearProgress = yearProgress - timeToNextEvent # update time to next year tracker
        timeCost = list(timeCost - np.repeat(timeToNextEvent, len(timeCost))) # update timecost tracker for every scientist
        timePeriod = timePeriod + timeToNextEvent # update the current model time
        if (timePeriod > lifeSpan):
            break
        
        if (min(timeCost) == 0): # studies are completed -> scientists store questions into their drawer
            if limit == False:
                yearProgress = 0
            # trackers for scientists who completed a study
            actingScientists = np.array([i for i, noTimeLeft in enumerate(timeCost) if noTimeLeft == 0]) # retrieves the indeces of people who have finished sampling
            amountActingScientists = len(actingScientists) # count amount of scientists who have completed their study
            questionActingScientists = list(scientistDataFrame[scientistDataFrame['scientistID'].isin(actingScientists)]['questionID']) # retrieve question IDs of questions that have been completed

            # store effect sizes and sample sizes of completed questions
            effectSizeQuestion = [] # create list for effect sizes of completed questions
            for i in questionActingScientists: # for every completed question index
                effectSizeQuestion.append(resultsDataFrame.loc[[i],'effectSize']) # store the effect size from the resultsDataFrame in the list of effect sizes of completed questions
            sampleSizeActingScientists = list(scientistDataFrame[scientistDataFrame['scientistID'].isin(actingScientists)]['sampleSize']) # list of sample sizes of completed questions


            # store statistical powers of the completed questions
            sampleSizeIndex = 0 # sample size index tracker
            powerOfQuestions = [] # list of powers of completed questions
            for qid in questionActingScientists: # for every completed question
                    resultsDataFrame.at[qid,'published'] = False
                    powerOfQuestion =  smp.ttest_power( effect_size= resultsDataFrame._get_value(qid, 'effectSize'),
                                                       nobs=sampleSizeActingScientists[sampleSizeIndex], 
                                                       alpha=0.05, alternative='two-sided') # calculate the power
                    powerOfQuestions.append(powerOfQuestion) # add power to list of powers of completed questions
                    sampleSizeIndex += 1 # increase sample size index tracker by one
            
            # simulate the result of a completed question based on the power
            positiveResult = [] # list of boolean results for completed questions
            for r in range(amountActingScientists): # for each completed question
                positiveResult.append(np.random.uniform(0,1) < powerOfQuestions[r]) # store positive or negative result based on power and a drawn value from a uniform distribution
            
            for sid in actingScientists: # for every acting scientist
                qid = questionActingScientists[np.where(actingScientists == sid)[0][0]] # retrieve the question ID they completed
                drawerQ[sid] = np.append(drawerQ[sid], [qid]) # add question ID to drawer
                drawerR[sid] = np.append(drawerR[sid], [positiveResult[np.where(actingScientists == sid)[0][0]]]) # add question result to drawer
            # update completed question tracker
            questionIDsWorked = np.concatenate((questionIDsWorked, questionActingScientists)) # add completed questions to the tracker
            
            # update the results dataframe 
            for sid in actingScientists: # for every scientist that completed a question
                qid = questionActingScientists[np.where(actingScientists == sid)[0][0]] # retrieve the question ID they completed
                resultsDataFrame.loc[[qid],'scientistID'] = sid # add scientist ID to results dataframe
                resultsDataFrame.loc[[qid],'sampleSize'] = sampleSizeActingScientists[np.where(actingScientists == sid)[0][0]] # add sample size to results dataframe
                resultsDataFrame.loc[[qid],'result'] = positiveResult[np.where(actingScientists == sid)[0][0]] # add result to results dataframe

            # find assignable questions and assign scientists to a new question
            for sid in actingScientists: # for every scientist
                nextQuestion = np.random.choice(unpubQ)
                if resultsDataFrame.loc[[qid],'effectSize'].item() > resultsDataFrame.loc[[nextQuestion],'effectSize'].item() and scientistDataFrame.loc[[sid],'chance'].item() != 0:
                    if  np.random.choice(scientistDataFrame.loc[[sid],'chance'].item(), 1) == 0:
                        nextQuestion = np.random.choice(unpubQ)
                        extraTimeCost[sid] = extraTimeCost[sid] + newQuestionCost
            
                scientistPerQuestion[qid] -= 1 # decrease number of scientist working on the completed question by one
                scientistPerQuestion[nextQuestion] += 1 # increase number of scientist working on the next question by one
                scientistDataFrame.loc[[
                    resultsDataFrame.loc[[qid],'scientistID'].item()], 'questionID'] = nextQuestion # update scientist dataframe with the new question ID

            # adjust the baseline
            for sid in actingScientists: # for every scientist
                timeCost[sid] = timeCostBaseline[sid] + extraTimeCost[sid]  # reset their time cost to the baseline value
            
            extraTimeCost = [0] * populationSize
                
        elif yearProgress == 0: # end of year -> scientists publish their papers
            yearProgress = oneYear # reset time to next year tracker
            
            for sid in np.repeat(scientistDataFrame['scientistID'], paperLimit): # for each scientist
                workedQ = drawerQ[sid] # retrieve all completed, unpublished questions from their drawer
                if len(workedQ) != 0: # if their drawer is not empty
                    payoff = 0 # initialize payoff
                    
                    # find question with highest payoff in a scientist's drawer and publish it
                    for q in range(len(workedQ)): # for each completed, unpublished question
                        priorPublished = questionIDsPublished.count(workedQ[q]) # count prior published works on this question
                        noveltyResult = pow( (1/(1+priorPublished)), scoopedCost) # calculate novelty of this question based on novelty and scooping cost
                        if drawerR[sid][q]: # if the result of the completed, unpublished question is positive 
                            possiblepayoff = noveltyResult # the possible payoff is equal to the payoff of a novel result
                        else: # if the result of the completed, unpublished question is negative
                            possiblepayoff = noveltyResult * negativeResultCost # the possible payoff is equal to the payoff of a novel result scaled down with the cost of a negative result
                        if possiblepayoff > payoff: # if a completed, unpublished question is found with a higher possible payoff than the current stored payoff
                            tempq = q # question to publish becomes the current completed, unpublished question
                            payoff = possiblepayoff # payoff becomes the highest possible payoff
                    payoff = np.round(payoff,2) # round payoff to two decimals
                    # publish question with the highest possible payoff
                    scientistDataFrame.at[sid, 'payoff'] = scientistDataFrame.at[sid, 'payoff'].astype(float) + payoff # add payoff to scientist's previous payoffs
                    scientistDataFrame.at[sid, 'publications'] = scientistDataFrame.at[sid, 'publications'].astype(int) + 1 # increase number of scientist's publications by one
                    questionIDsPublished.append(drawerQ[sid][tempq]) # add now published question to the list of published questions
                    resultsDataFrame.at[drawerQ[sid][tempq],'published'] = True # change published status to True
                    
                    drawerP.append([resultsDataFrame.loc[[drawerQ[sid][tempq]],'effectSize'].item(),drawerR[sid][tempq]])
                    
                    # remove the completed, published question from the drawer of completed, unpublished questions
                    if drawerQ[sid][tempq] in unpubQ:
                        unpubQ.remove(drawerQ[sid][tempq])
                    drawerQ[sid] = np.delete(drawerQ[sid], tempq) # remove question from drawer with question IDs
                    drawerR[sid] = np.delete(drawerR[sid], tempq) # remove question from drawer with question results
                    
    
    for sid in scientistDataFrame['scientistID']:
        for q in range(len(drawerQ[sid])):
            drawerU.append([resultsDataFrame.loc[[drawerQ[sid][q]],'effectSize'].item(),drawerR[sid][q]])
        
    publishedDataFrame = pd.DataFrame(data=drawerP, columns = ['effectSize','result'])
    unpublishedDataFrame = pd.DataFrame(data=drawerU, columns = ['effectSize','result'])
    finishedDataFrame =  resultsDataFrame[(resultsDataFrame['published'] == True) | (resultsDataFrame['published'] == False)].reset_index() # store all finished questions in dataframe  
          

    return scientistDataFrame , resultsDataFrame, finishedDataFrame, drawerQ, publishedDataFrame, unpublishedDataFrame  # return scientist dataframe

In [ ]:
# run generations of scientists that compete against each other and evolve
def evolution(lifeSpan, scientistPerQuestionMax, generations, startupCost, sampleCost, ExpDistributionShape, scoopedCost, negativeResultCost, limit, newQuestionCost, randomChance, chance):
    
    for run in range(10):
        print('Run: '+ str(run))
        
        # initialize population
        sampleSize = np.round(np.random.uniform(minSampleSize, maxSampleSize, populationSize)) # draw sample sizes from a uniform distribution
        meanSampleSizes = []   # list of mean sample sizes
        meanPayoffs = []       # list of mean payoffs
        meanPublished = []     # list of mean number of published questions
        maxQ = []             # list of mean question IDs
        meanChance = []
        percFalsePosAll = []   # list of percentages of false positives (all finished questions)
        percFalsePosPub = []   # list of percentages of false positives (all published questions)
        percFalsePosUnpub = [] # list of percentages of false positives (all unpublished questions)
        percFalseNegAll = []   # list of percentages of false negatives (all finished questions)
        percFalseNegPub = []   # list of percentages of false negatives (all published questions)
        percFalseNegUnpub = [] # list of percentages of false negatives (all unpublished questions)
        generationsPlot = (np.arange(generations)) + 1 
        drawerSizes = []
        percPosPublished = []
        trackFalsePos = []
        trackFalseNeg = []
        trackFalsePosPub = []
        trackFalseNegPub = []


        for g in range(generations): # for every generation 
            print("working on generation: ",g+1,"/",generations)
            result = competition(lifeSpan, sampleSize, scientistPerQuestionMax, startupCost, sampleCost, 
                                          ExpDistributionShape, scoopedCost, negativeResultCost, limit, newQuestionCost, randomChance, chance) # return scientist dataframe
            outcomeGeneration = result[0]
            # allocate new generation of scientists a sample size with payoffs of their predecessors as (probability) weights
            sampleSize = outcomeGeneration.sample(n=populationSize, weights='payoff', random_state=1, replace= True)['sampleSize'].to_numpy() 
            chance = outcomeGeneration.sample(n=populationSize, weights='payoff', random_state=1, replace= True)['chance'].to_numpy()

            for s in range(len(sampleSize)): # for each sample size
                sampleSize[s] = np.absolute(np.round(np.random.normal(sampleSize[s], 1.5, 1))) # introduce noise

            meanSampleSizes.append(outcomeGeneration["sampleSize"].mean()) # add mean sample size to list of mean sample sizes
            meanPayoffs.append(outcomeGeneration["payoff"].mean())         # add mean payoff to list of mean payoffs
            meanPublished.append(outcomeGeneration["publications"].mean()) # add mean number of published questions to list of mean number of published questions
            maxQ.append(outcomeGeneration["questionID"].max())           # add mean question ID to list of mean question IDs
            meanChance.append(outcomeGeneration["chance"].mean())

            finishedDataFrame = result[2] # retrieve finished questions dataframe

            published = result[4]
            #print(published)
            unpublished = result[5]
            frames = [published, unpublished]
            allResearched = pd.concat(frames)

            # calculating false positive and false negative rates for all finished questions
            lenFinished = len(allResearched) # number of finished questions
            falsePosAll = (allResearched['effectSize'] == 0.0) & (allResearched['result'] == True) # false positive boolean mask
            falseNegAll = (allResearched['effectSize'] > 0.0) & (allResearched['result'] == False) # false negative boolean mask
            nrFalsePosAll = allResearched[falsePosAll].count()[0] # number of false positives
            nrFalseNegAll = allResearched[falseNegAll].count()[0] # number of false negatives
            percFalsePosAll.append((nrFalsePosAll/lenFinished)*100) # add false positive percentage to list of false positives
            percFalseNegAll.append((nrFalseNegAll/lenFinished)*100) # add false negative percentage to list of false negatives

            # calculating false positive and false negative rates for all published questions
            lenFinishedPub = len(published) # number of published questions
            falsePosPub = (published['effectSize'] == 0.0) & (published['result'] == True) # false positive boolean mask
            falseNegPub = (published['effectSize'] > 0.0) & (published['result'] == False) # false positive boolean mask # false negative boolean mask
            nrFalsePosPub= published[falsePosPub].count()[0] # number of false positives
            nrFalseNegPub = published[falseNegPub].count()[0] # number of false negatives
            percFalsePosPub.append((nrFalsePosPub/lenFinishedPub)*100) # add false positive percentage to list of false positives
            percFalseNegPub.append((nrFalseNegPub/lenFinishedPub)*100) # add false negative percentage to list of false negatives

            # calculating false positive and false negative rates for all unpublished questions
            lenFinishedUnpub = len(unpublished) # number of unpublished questions
            falsePosUnpub = (unpublished['effectSize'] == 0.0) & (unpublished['result'] == True) # false positive boolean mask
            falseNegUnpub = (unpublished['effectSize'] > 0.0) & (unpublished['result'] == False) # false negative boolean mask
            nrFalsePosUnpub = unpublished[falsePosUnpub].count()[0] # number of false positives
            nrFalseNegUnpub = unpublished[falseNegUnpub].count()[0] # number of false negatives
            if lenFinishedUnpub > 0:
                percFalsePosUnpub.append((nrFalsePosUnpub/lenFinishedUnpub)*100) # add false positive percentage to list of false positives
                percFalseNegUnpub.append((nrFalseNegUnpub/lenFinishedUnpub)*100) # add false negative percentage to list of false negatives
            else:
                percFalsePosUnpub.append(0) # add false positive percentage to list of false positives
                percFalseNegUnpub.append(0) # add false negative percentage to list of false negatives

            #printProgressNew = outcomeGeneration[['sampleSize','payoff', 'publications']] 
            #printProgress = pd.concat([printProgress, printProgressNew], axis=1)
            #print(outcomeGeneration)

            trackFalsePos.append(nrFalsePosAll)
            trackFalseNeg.append(nrFalseNegAll)
            trackFalsePosPub.append(nrFalsePosPub)
            trackFalseNegPub.append(nrFalseNegPub)

            positives = np.sum(published['result'] == True)
            #print(positives)
            resultsTotal = len(published)
            if resultsTotal != 0:
                percPosPublished.append(100 * np.round((positives / resultsTotal), 2))
            else:
                percPosPublished.append(0)

            drawerSize = 0
            drawer = result[3]
            for i in range(populationSize):
                drawerSize += len(drawer[i])
            drawerSizes.append(drawerSize)

            ##outcomeGeneration.to_csv('file' + str(g) +'.csv')


        # plot mean payoff, mean sample size, mean number of published questions and mean question ID against generations
        fig, axs = plt.subplots(1,2)

        fig.suptitle('Progression with limit set to {}'.format(limit))
        axs[0].plot(generationsPlot, meanPayoffs, 'tab:orange')
        axs[0].set_title('Mean Payoffs')
        axs[0].set(ylim=(0, max(meanPayoffs)+max(meanPayoffs)/10))

        axs[1].plot(generationsPlot, meanSampleSizes,  'tab:green')
        axs[1].set_title('Mean SampleSize')
        axs[1].set(ylim=(0, max(meanSampleSizes)+max(meanSampleSizes)/10))

        fig, axs = plt.subplots(1,2)
        fig.suptitle( "Progression, limit = {}".format(limit) )

        axs[1].plot(generationsPlot, meanPublished, 'tab:red')
        axs[1].set_title('Mean Published')
        axs[1].set(ylim=(0, max(meanPublished)+max(meanPublished)/10))

        axs[0].plot(generationsPlot, maxQ, 'tab:orange')
        axs[0].set_title('Questions worked on')
        axs[0].set(ylim=(0, round((lifeSpanT / (startupCostT + minSampleSize)) * populationSize + populationSize)))

        fig, axs = plt.subplots(1,2)
        fig.suptitle('All finished questions, limit = {}'.format(limit) )
        axs[0].plot(generationsPlot, percFalsePosAll, 'tab:green')
        axs[0].set_title('Percentage False Positives')
        axs[0].set(ylim=(0, 101))

        axs[1].plot(generationsPlot, percFalseNegAll, 'tab:red')
        axs[1].set_title('Percentage False Negatives')
        axs[1].set(ylim=(0, 101))

        fig, axs = plt.subplots(1,2)
        fig.suptitle('All published questions, limit = {}'.format(limit))
        axs[0].plot(generationsPlot, percFalsePosPub, 'tab:green')
        axs[0].set_title('Percentage False Positives')
        axs[0].set(ylim=(0, 101))

        axs[1].plot(generationsPlot, percFalseNegPub, 'tab:red')
        axs[1].set_title('Percentage False Negatives')
        axs[1].set(ylim=(0, 101))

        fig, axs = plt.subplots(1,2)
        fig.suptitle('All unpublished questions, limit = {}'.format(limit))
        axs[0].plot(generationsPlot, percFalsePosUnpub, 'tab:green')
        axs[0].set_title('Percentage False Positives')
        axs[0].set(ylim=(0, 101))

        axs[1].plot(generationsPlot, percFalseNegUnpub, 'tab:red')
        axs[1].set_title('Percentage False Negatives')
        axs[1].set(ylim=(0, 101))

        fig, axs = plt.subplots(1,2)
        fig.suptitle('File-drawer, limit = {}'.format(limit))
        axs[0].plot(generationsPlot, drawerSizes, 'tab:red')
        axs[0].set_title('Papers file-drawer')
        axs[0].set(ylim=(0, max(drawerSizes)+10))

        axs[1].plot(generationsPlot, percPosPublished, 'tab:red')
        axs[1].set_title('% positive published results')
        axs[1].set(ylim=(0, 101))

        fig, axs = plt.subplots(1,2)

        fig.suptitle('Mean chace'.format(limit))
        axs[0].plot(generationsPlot, meanChance, 'tab:orange')
        axs[0].set_title('Mean chace')
        axs[0].set(ylim=(0, max(meanChance)+max(meanChance)/10))

        d5 = {'FP all': trackFalsePos, 'FN all' : trackFalseNeg, 'FP pub': trackFalsePosPub, 'FN pub': trackFalseNegPub }
        falseDataframe = pd.DataFrame(data=d5)
        print(falseDataframe)


        meanSampleSizeOverRuns.append(meanSampleSizes)
        meanPayoffsOverRuns.append(meanPayoffs)
    

In [ ]:
meanSampleSizeOverRuns = []
meanPayoffsOverRuns = []

In [ ]:
#number of scientists
populationSize = 120

#Scientists initial sample sizes 
minSampleSize = 2
maxSampleSize = 1000

# T = Test variable
lifeSpanT = 15000
scientistPerQuestionMaxT = 1
startupCostT = 125
sampleCostT = 1
ExpDistributionShapeT = 8
scoopedCostT = 0.5
negativeResultCostT = 0.5
paperLimit = 1
limit = True
newQuestionCost = 0
randomChance = False
chance = 0

generationsT = 100

# for x in range(2):
#     print("limit is {}".format(limit))
#     evolution (lifeSpanT, scientistPerQuestionMaxT, generationsT, startupCostT, sampleCostT, ExpDistributionShapeT, scoopedCostT, negativeResultCostT, limit)
#     limit = not limit

In [ ]:
%%time
limit = True
evolution (lifeSpanT, scientistPerQuestionMaxT, generationsT, startupCostT, sampleCostT, ExpDistributionShapeT, scoopedCostT, negativeResultCostT, limit, newQuestionCost, randomChance, chance)

In [ ]:
%%time
limit = False
evolution (lifeSpanT, scientistPerQuestionMaxT, generationsT, startupCostT, sampleCostT, ExpDistributionShapeT, scoopedCostT, negativeResultCostT, limit, newQuestionCost, randomChance, chance)

In [ ]:
meanSamplesDataFrame = pd.DataFrame(data = meanSampleSizeOverRuns)
print(meanSamplesDataFrame)
meanPayoffDataFrame = pd.DataFrame(data = meanPayoffsOverRuns)
print(meanPayoffDataFrame)
meanSamplesDataFrame.to_csv('meanSamplesDataFrame.csv')
meanPayoffDataFrame.to_csv('meanPayoffDataFrame.csv')